# Longest Prefix Matching Tokenization

Applied to Thai Language

## 1. Load file

In [0]:
import csv
def load_csv(path):
    with open(path, "r") as file:
        reader = csv.reader(file)
        return_list = []
        for row in reader:
            return_list.append(row[0])
    return return_list

In [240]:
THAI_CORPUS = load_csv("thai-corpus-from-NECTEC.csv")
THAI_CORPUS[:10]

['ก.',
 'ก.ค.',
 'ก.ต.',
 'ก.ป.ส.',
 'ก.พ.',
 'ก.พ.ด.',
 'ก.ม.',
 'ก.ย',
 'ก.ย.',
 'ก.ร.']

## 2. Create corpus index -> optimize time complexity

In [0]:
# Peek the corpus, look at the first character, and record its index
def create_corpus_index(corpus): 
    index_list = []
    curr_char = corpus[0][0]
    cnt = 0
    for idx, word in enumerate(corpus):
        if idx == 0: 
            j = 0
            while word[0] == word[j]:
                j = j+1
            index_list.append([word[0], 0, j+1])
        elif idx == len(corpus)-1:
            index_list.append({"alphabet": curr_char,
                               "start": idx-cnt-1,
                               "end": idx+1})
        else:
            last_char = curr_char
            curr_char = word[0]
            if last_char != curr_char:
                index_list.append({"alphabet": last_char,
                                   "start": idx-cnt-1,
                                   "end": idx})
                cnt = 0
            else:
                cnt = cnt+1
    return index_list[1:]  

In [242]:
CORPUS_INDEX = create_corpus_index(THAI_CORPUS)
CORPUS_INDEX[:10]

[{'alphabet': 'ก', 'end': 3425, 'start': 0},
 {'alphabet': 'ข', 'end': 4504, 'start': 3425},
 {'alphabet': 'ฃ', 'end': 4505, 'start': 4504},
 {'alphabet': 'ค', 'end': 7059, 'start': 4505},
 {'alphabet': 'ฅ', 'end': 7060, 'start': 7059},
 {'alphabet': 'ฆ', 'end': 7072, 'start': 7060},
 {'alphabet': 'ง', 'end': 7274, 'start': 7072},
 {'alphabet': 'จ', 'end': 8080, 'start': 7274},
 {'alphabet': 'ฉ', 'end': 8230, 'start': 8080},
 {'alphabet': 'ช', 'end': 9112, 'start': 8230}]

## 3. Word checking function

In [0]:
def isWord(input_string, corpus, corpus_index):
    if input_string == None: # input_string is none
        return False
    for index_obj in corpus_index:
        try: # Check only first character with the corpus index
            if index_obj["alphabet"] == input_string[0]:
                # Narrow the scope of checking
                corpus_temp = corpus = corpus[index_obj["start"]:index_obj["end"]]
                if(input_string in corpus_temp): 
                    return True
                else:
                    return False
        except:
            # The first character will never provide to become a word
            # For instance, Thai consonants or some vowels
            return False
    return False

In [244]:
print(isWord("ตากลม", THAI_CORPUS, CORPUS_INDEX))
print(isWord("ตากม", THAI_CORPUS, CORPUS_INDEX))
print(isWord("ตกลม", THAI_CORPUS, CORPUS_INDEX))

True
False
False


## 4. Tokenizer

In [0]:
def lpm_tokenize(input_text, corpus, corpus_index):
    st = 0 # word start pointer
    end = len(input_text) + 1 # word end pointer
    post_st = -1 # last word start pointer
    post_end = -1 # last word end pointer
    go_back = False # backward flag
    found_misspell = False # undefined word flag
    word_list = [] # word list
    check_point = None # word list before append a current word
    word = input_text[st:end] # initial word

    while(len(input_text[st:end]) != 0):
        while((isWord(input_string=word, corpus=corpus, corpus_index=corpus_index) == False) and (found_misspell == False)):
            end = end-1 # end pointer be decreased til a word is found
            if(end < st): # no word is found -> unfit occur
                go_back = True
                break
            word = input_text[st:end] # repeat checking

        if (go_back == False): # normal case: found a word -> set new pointers 
            post_st = st
            post_end = end
            st = end
            end = len(input_text)+1
            word_list.append(word) # append that word into a returnned list
            check_point = word_list.copy() # save check point list

        elif (go_back == True): # select the second longest
            if (len(word_list) == 0): # condition: go back til no word left
                if check_point == None: break # input text is not a word
                word_list = check_point.copy() # load the check point
                found_misspell = True # set a misspell flag
                print("WARNING: WORD IS NOT IN CORPUS, TOKENIZING STOPPED.")
                break
            word_list.pop() # unfit occur -> pop last word
            st = post_st # set pointer back
            end = post_end-1
            go_back = False # then go check is it word again
        word = input_text[st:end]
    return word_list

In [246]:
print(lpm_tokenize("ตากลมนั่งตากลม", THAI_CORPUS, CORPUS_INDEX))
print(lpm_tokenize("ตตากลม", THAI_CORPUS, CORPUS_INDEX))
print(lpm_tokenize("ตม", THAI_CORPUS, CORPUS_INDEX))

['ตากลม', 'นั่ง', 'ตากลม']
[]
['ตม']


## 5. Testing

In [0]:
testing_text = "เมื่อสุนัขตกจากสะพานชำรุดสู่ลำน้ำมันรีบว่ายขึ้นฝั่งเชิดหน้าสะบัดขนไม่บ่นว่าหรือด่าทอสะพานและก็ไม่ร้องขานแก้ตัวไม่มีสัตว์โลกชนิดใดเลยที่จะแก้ตัวให้กับความผิดพลาดของมันมันจะทำในสิ่งที่เหมาะสมกว่านั่นคือเริ่มต้นใหม่อย่างไม่ประมาทเมื่อบริสุทธิ์อย่างแท้จริงทำไมต้องพะวงกับการกล่าวหายามผิดพลาดอย่าพะวง"

In [252]:
import time

start = time.time()
print(lpm_tokenize(testing_text, THAI_CORPUS, INDEX_CORPUS))
end = time.time()
time = end - start
print(str(time) + " second")

['เมื่อ', 'สุนัข', 'ตก', 'จาก', 'สะพาน', 'ชำรุด', 'สู่', 'ลำน้ำ', 'มัน', 'รีบ', 'ว่าย', 'ขึ้นฝั่ง', 'เชิดหน้า', 'สะบัด', 'ขน', 'ไม่', 'บ่นว่า', 'หรือ', 'ด่าทอ', 'สะพาน', 'และ', 'ก็', 'ไม่', 'ร้อง', 'ขาน', 'แก้ตัว', 'ไม่มี', 'สัตว์โลก', 'ชนิด', 'ใด', 'เลย', 'ที่จะ', 'แก้ตัว', 'ให้', 'กับ', 'ความผิดพลาด', 'ของ', 'มัน', 'มัน', 'จะ', 'ทำ', 'ใน', 'สิ่ง', 'ที่', 'เหมาะสม', 'กว่า', 'นั่น', 'คือ', 'เริ่มต้น', 'ใหม่', 'อย่าง', 'ไม่', 'ประมาท', 'เมื่อ', 'บริสุทธิ์', 'อย่างแท้จริง', 'ทำไม', 'ต้อง', 'พะวง', 'กับ', 'การกล่าวหา', 'ยาม', 'ผิดพลาด', 'อย่า', 'พะวง']
0.32450079917907715 second
